# PSO ALGORITHM

In [6]:
from softpy.evolutionary.candidate import FloatVectorCandidate
from typing import Callable
import numpy as np
import random

In [ ]:
class ParticleCandidate(FloatVectorCandidate):
    """
    A candidate for particle swarm optimization (PSO) that represents a particle
    in the swarm with position and velocity attributes.
    """
    
    def __init__(self, size: int, lower: np.ndarray, upper: np.ndarray, candidate: np.ndarray, velocity: np.ndarray, 
                 inertia: float, w_l: float, w_n: float, w_g: float):
        # Call parent constructor
        super().__init__(size, candidate, None, lower=lower[0], upper=upper[0], intermediate=False, mutate=None, recombine=None)
        
        if w_l + w_n + w_g != 1.0:
            raise ValueError("w_l + w_n + w_g must equal 1.0")
        
        # Initialize particle-specific attributes
        self.size = size
        self.lower = lower.astype(float)
        self.upper = upper.astype(float)
        self.candidate = candidate.astype(float)
        self.velocity = velocity.astype(float)
        self.inertia = inertia
        self.w_l = w_l
        self.w_n = w_n
        self.w_g = w_g

    def generate(size: int, lower: np.ndarray, upper: np.ndarray, inertia: float, 
                 w_l: float, w_n: float, w_g: float):
        # Generate a random candidate within the bounds  
        candidate = np.random.uniform(lower, upper, size)
        
        # Generate velocity within the specified range
        velocity_range = np.abs(upper - lower)
        velocity = np.random.uniform(-velocity_range, velocity_range, size)
        
        # Create and return particle instance
        return ParticleCandidate(size, lower, upper, candidate, velocity, inertia, w_l, w_n, w_g)
    
    def mutate(self):
        # Update candidate position by adding velocity
        self.candidate += self.velocity
    
    def recombine(self, local_best, neighborhood_best, best):
        """Update velocity according to PSO formula"""
        r_l = random.uniform(0, 1)
        r_n = random.uniform(0, 1)
        r_g = random.uniform(0, 1)
        
        self.velocity = (
            self.inertia * self.velocity
            + r_l * self.w_l * (local_best.candidate - self.candidate)
            + r_n * self.w_n * (neighborhood_best.candidate - self.candidate)
            + r_g * self.w_g * (best.candidate - self.candidate)
        )

In [ ]:
class ParticleSwarmOptimizer(MetaHeuristicsAlgorithm):